# `Trotter Costs`

We want to estimate the cost of implementing time evolution of a wavefunction:

$$
|\psi(t)\rangle = e^{-i H t}|\psi(0)\rangle
$$

fault tolerantly using qualtran. The time evolution unitary can be implemented using Suzuki-Trotter methods, that is, if
$$
H = H_1 + H_2,
$$
we can always write
$$
e^{-i H t} = \lim_{n\rightarrow\infty} \left(e^{-iH_1 t/m} e^{-iH_2 t/m}\right)^{m}
$$
Trotter methods exploit this identity to approximate the exact dynamics through product formulae. The error in these approximations is related to higher order commutators of the components of the Hamiltonian. For example, the simplest Trotter approximation allows us to approximate the unitary as
$$
e^{-i H t} = \prod_m^{N_t} e^{-iH_1 \delta t} e^{-iH_2 \delta t} + \mathcal{O}(\delta t^2)
$$
where we have $\delta_t = t / (N_t + 1)$. More sophisticated trotter breakups lead to better accuracy, at the cost of more complicated product formula.

For the ab-initio chemistry Hamiltonian (in atomic units) we have

\begin{align}
H &= -\frac{1}{2} \sum_i \nabla_i^2 -\sum_{i}\sum_{J} \frac{\zeta_J}{|R_J-r_i|} + \sum_{i < j} \frac{1}{|r_i-r_j|} \\
  &= T + U + V
\end{align}

Note that the Coulomb terms are diagonal in the position basis while the kinetic term is diagonal in the momentum basis. Thus we can employ a QFT: 
$$
|\psi(t)\rangle \approx \mathrm{QFT} e^{-i\delta t T} \mathrm{QFT}^{\dagger} e^{-i\delta t U}  e^{-i \delta t V} |\psi(0)\rangle
$$
so that all the terms can be implemented via a gate which implements something of the form $e^{-i \delta t \phi_\alpha }$ via a phasing gate.


## ```First Quantized Grid Based Hamiltonian```

We are concerned with implementing time evolution in the first quantized representation using a grid based Hamiltonian. Our wavefunction will be represented on a real space grid 

$$
|\psi\rangle = \sum_{r_1\cdots r_\eta} c(r_1, \cdots, r_\eta) |r_1\cdots r_\eta\rangle
$$

and each $r$ lives on a grid of size $N = (2 N_g + 1)^3$ if $N_g$ is the number of grid points in each spatial dimension. Thus we have $\eta$ registers of size $n = 3 \lceil \log N^{1/3}\rceil + 1$.

According to Jones et al., the main steps to implement are (for the electron-electron interaction $V$, but the other terms are similar)

\begin{align}
&\sum_{r_1\cdots r_\eta} c(r_1, \cdots, r_\eta) |r_1\cdots r_\eta\rangle \\
  &\rightarrow  \sum_{r_1\cdots r_\eta} c(r_1, \cdots, r_\eta) |r_1\cdots r_\eta\rangle|V(r_1\cdots r_\eta)\rangle \hspace{10em} \text{Compute pairwise potential in ancilla registers} \\
  &\rightarrow \sum_{r_1\cdots r_\eta} e^{-i \delta t V(r_1\cdots r_\eta)} c(r_1, \cdots, r_\eta) |r_1\cdots r_\eta\rangle|V(r_1\cdots r_\eta)\rangle \hspace{5.5em} \text{Phase the state with computed potential} \\ 
  &\rightarrow \sum_{r_1\cdots r_\eta} e^{-i \delta t V(r_1\cdots r_\eta)} c(r_1, \cdots, r_\eta) |r_1\cdots r_\eta\rangle|0\cdots0\rangle \hspace{7.8em} \text{Uncompute potential in ancilla register} \\ 
\end{align}
in the above the ancilla register storing the value of the potential is of size.

To compute the potential we need to compute $\frac{1}{r_{ij}} \equiv \frac{1}{|r_i - r_j|}$ which can be done in two steps:

1. compute $|r_{ij}^2\rangle = |(x_i - x_j)^2 + (y_i-y_j)^2 + (z_i - z_j)^2\rangle$ into a register of size $2 n + 2$.
2. compute the inverse square root of the number in this register ($r_{ij}^2$). The cost of computing the sum of squares of the electronic positions requires computing the 3 subtractions at a cost of approximately 3n Toffolis, and the sum of 3 squares which has a cost of $3n^2 - n - 1$ Toffolis.

To compute $\frac{1}{r_{ij}}$ we solve $x^{-2} = r_{ij}^2$ which has solution $x^* = \frac{1}{r_{ij}}$. This can be solved iteratively using the Newton-Raphson method: 

\begin{align}
a_{n+1} &= a_{n} - \frac{a_n^3(1-r_{ij}^2 a_n^2)}{-2 a_n^2} \\
        &= \frac{1}{2}a_n\left(3-a_n^2 r_{ij}^2\right).
\end{align}

In the fusion paper this method is improved upon by using a hybrid approach based upon QROM function interpolation in conjunction with this Newton-Raphson iteration. A further optimization is introduced to include the scaling factors necessary for the potential directly in the Newton-Raphson step (i.e. all the timestep and factors of two.)



## ```Function interpolation```

The basic idea is to first fit a cubic polynomial to $\frac{1}{r_{ij}}$ in a piecewise fashion over an interval of size 1. We can obtain the function at points outside the interval by scaling the polynomial coefficients by an appropriate power of 2. We will use variably spaced QROM to output the scaled polynomial coefficients within the range we want to evaluate our function at, after which we can evaluate the polynomial using arithmetic (multiplications and additions / subtractions) 

As an example consider approximating $\frac{1}{\sqrt{x}}$ in the range [1, 3/2]. The fusion paper provides the following approximation

$$
\frac{1}{\sqrt{x}} \approx a_0 - a_1 (x-1) + a_2 (x-1)^2 - a_3 (x-1)^3
$$

let's see how this compares: 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def polynomial_approx_range_one(x: float):
    a0 = 0.99994132489119882162
    a1 = 0.49609891915903542303
    a2 = 0.33261112772430493331
    a3 = 0.14876762006038398086
    return a0 - (x-1) * (a1 - (x-1) * (a2 - a3*(x-1)))


xs = np.linspace(1, 1.5, 10)
plt.plot(xs, 1.0 / xs**0.5, ls=":", label="exact")
plt.plot(xs, polynomial_approx_range_one(xs), marker='o', label="polyfit", lw=0)
plt.xlabel("$x$")
plt.ylabel("$1/\sqrt{x}$")
plt.legend()
plt.show()
delta = np.max(np.abs(polynomial_approx_range_one(xs) - xs**(-0.5)))
print(f"max error = {delta}")

Ok, that's pretty good, but the author's of the fusion paper now combine this with a step of Newton-Raphson to improve the accuracy. Let's see if that's the case:

In [ ]:
def newton_raphson_step_range_one(x, y0):
    # This odd delta shift let's us reduce the error by about a factor of two.
    # Try commenting it out and see what the max error is after this step.
    delta = 5.1642030908180720584e-9
    yprime = 0.5 * y0 * (3 + delta - y0**2 * x) 
    return yprime

poly_fit = polynomial_approx_range_one(xs)
newton_update = newton_raphson_step_range_one(xs, poly_fit)
delta = np.max(np.abs(newton_update - xs**(-0.5)))
print(f"max error after Newton-Raphson = {delta}")

Perf! Now how do we obtain the value of the function in say the range [16, 23], well this is just $16 = 2^4$ times the range of [1, 3/2], thus we can appropriately scale our polynomial coefficients by a factor of $2^4$. Let's check this:

In [ ]:
def polynomial_approx_range_one_scaling(x: float, scale_power: int):
    a0 = 0.99994132489119882162 / 2**(scale_power/2.0)
    a1 = 0.49609891915903542303 / 2**(3*scale_power/2.0)
    a2 = 0.33261112772430493331 / 2**(5*scale_power/2.0)
    a3 = 0.14876762006038398086 / 2**(7*scale_power/2.0)
    return a0 - (x-2**scale_power) * (a1 - (x-2**scale_power) * (a2 - a3*(x-2**scale_power)))

xs_new = 2**4 * xs
poly_fit = polynomial_approx_range_one_scaling(xs_new, scale_power=4)
newton_update = newton_raphson_step_range_one(xs_new, poly_fit)
delta = np.max(np.abs(newton_update - xs_new**(-0.5)))
print(f"max error after Newton-Raphson = {delta}")

For completeness, the second polynomial interpolation for the range $[3/2, 2]$

In [ ]:
def polynomial_approx_range_two_scaling(x: float, scale_power: int=0):
    b0 = 0.81648515205385221995 / 2**(scale_power/2.0)
    b1 = 0.27136515484240234115 / 2**(3*scale_power/2.0)
    b2 = 0.12756148214815175348 / 2**(5*scale_power/2.0)
    b3 = 0.044753028579153842218 / 2**(7*scale_power/2.0)
    return b0 - (x-1.5*2**scale_power) * (b1 - (x-1.5*2**scale_power) * (b2 - b3*(x-1.5*2**scale_power)))

def newton_raphson_step_range_two(x, y0):
    # This odd delta shift let's us reduce the error by about a factor of two.
    # Try commenting it out and see what the max error is after this step.
    delta = 3.6279794522852781448e-10 
    yprime = 0.5 * y0 * (3 + delta - y0**2 * x) 
    return yprime

xs = np.linspace(1.5, 2.0, 10)
poly_fit = polynomial_approx_range_two_scaling(xs)
newton_update = newton_raphson_step_range_two(xs, poly_fit)
delta = np.max(np.abs(newton_update - xs**(-0.5)))
print(f"max error after Newton-Raphson = {delta}")
plt.plot(xs, np.abs(1.0 / xs**0.5 - polynomial_approx_range_two_scaling(xs)), marker='o', label="Polynomial", lw=0)
plt.plot(xs, np.abs(1.0 / xs**0.5 - newton_raphson_step_range_two(xs, polynomial_approx_range_two_scaling(xs))), marker='x', label="Newton-Raphson", lw=0)
plt.xlabel("$x$")
plt.ylabel("Error")
plt.yscale("log")
plt.legend()

In [ ]:
# Check the polynomial outside the range (2**5 * [1.5, 2)
xs_new = 2**5 * xs 
poly_fit = polynomial_approx_range_two_scaling(xs_new, scale_power=5)
newton_update = newton_raphson_step_range_two(xs_new, poly_fit)
delta = np.max(np.abs(newton_update - xs_new**(-0.5)))
print(f"max error after Newton-Raphson = {delta}")

Ok! Our polynomial interpolation works, and we can obtain a very accurate approximation for $r_{ij}^{-1}$ using two polynomials (i.e. 8 coefficients and two shift factor ($2^m$ or $1.5\times 2^m$))
Thus, we can use QROM to output our $2\times
m$ scaled polynomial coefficients, followed by three multiplications at a cost of roughly $3n^2$, Toffolis. There is a further optimization here in that we can use a `variable spaced` QROM to reduce the QROM cost substantially by only iterating over our integer intervals, this reduces the complexity to $g-1$ Toffolis where $g$ is the number of distinct intervals, which is a significant reduction of the typical QROM cost of $L-1$, where $L$ is the iteration length.

## ```Variable Spaced QROM```

Recall that QROM implements
$$
\mathrm{QROM}_d \sum_l \alpha_l |l\rangle|0\rangle = \sum_l \alpha_l |l\rangle|d_l\rangle.
$$
That is, given a selection register $l$, QROM can load the binary representation of the $l$-th data set $d$ into a target register of a given size.
In our case $|r_{ij}^2\rangle$ is the selection register of size $2 n + 2$, and we want to load $b$-bit binary representations of our polynomial coefficients $\{a_0, a_1, a_2, a_3\}$ and $\{b_0, b_1, b_2, b_3\}$ for the two ranges for each $m$. In this scheme we separate out our iteration into distinct parts, and we increment $[l, l + 2^n]$, where $n$ is incremented when it is possible ignore the least significant bits of the selection register. For example, for n = 5, $L=2^4=16$, the integer ranges are 0, 1, 2, 3, [4, 5], [6, 7], [8, 11], [12, 15]. This can be understood by considering the `unary iteration` circuit which is used during QROM construction. There, depending on the particular binary representation of the selection register different parts of the tree are traversed before writing the data to a register. We can delete parts of our unary iteration circuit where we want to repeat the data for a range of integers. The number of allowed integers in each range is determined in the following way: for the starting integer of the range $l$: if $k$ is the most significant bit of $l$ (where $l$ is restricted to be a power of 2), then the number of integers in our range is $2^{k-2}$ (we only bin integers when $k \ge 2$). Here is some code to generate the ranges:

In [ ]:
nbits = 6
nbits_rij_sq = 2 * nbits + 2
print(f"L = {2**nbits}")
mod_fac = 1
hit = True 
g = 0
for l in range(0, 2**nbits_rij_sq):
    k = l.bit_length()
    if k > 2:
        if l & (1 << (k-2)) and not hit:
            hit = True
            print(f"l = {l:0{nbits_rij_sq}b}, range = [{l:5d}, {l+2**(k-2)-1:5d}], length = {2**(k-2):>4d}")
            g += 1
        if not l & (1 << (k - 2)) and hit:
            hit = False
            print(f"l = {l:0{nbits_rij_sq}b}, range = [{l:5d}, {l+2**(k-2)-1:5d}], length = {2**(k-2):>4d} ")
            g += 1
# Add four to account for 0, 1, 2, 3
print(f"number of distinct regions g = {g + 4}. Toffoli cost g - 2 = {g - 2 + 4}")

We see by the bit pattern that we can use the $k-1$ st bit to toggle between our two ranges [1, 3/2] and [3/2, 2]. The total Toffoli cost is then just g - 2, which is 26 if $r_{ij}^2$ is stored with 14 bits. Note we can store the coefficients to high precision as there is only a small clifford overhead associated with this. In the fusion paper 15 bits of precision is suggested as sufficient. 

# Kinetic Energy Bloq
Recall that for a grid based Hamiltonian we represent our state on a real space grid of size $N$ where $x,y,z \in [-(N^{1/3}-1)/2, (N^{1/3}-1)/2]$. 

In [ ]:
for n in range(3, 21): 
    num_grid_total = (2*n+1)**3
    num_grid_each_dim = int((np.ceil(num_grid_total**(1/3)) - 1)//2)
    num_bits_each_dim = int(np.ceil(np.log2(np.ceil(num_grid_total**(1/3.0))))) + 1 
    assert num_bits_each_dim == (2*n + 1 - 1).bit_length() + 1

In [ ]:
from qualtran.bloqs.chemistry.trotter import KineticEnergy 
from qualtran.drawing import show_bloq
num_elec = 2
num_grid_each_dim = 2*10 + 1
ke_bloq = KineticEnergy(num_elec, num_grid_each_dim)
cbloq = ke_bloq.decompose_bloq()
show_bloq(ke_bloq.decompose_bloq())

# Potential Energy Bloq



In [ ]:

from qualtran.bloqs.chemistry.trotter import PotentialEnergy 
from qualtran.drawing import show_bloq
num_elec = 3
num_grid_each_dim = 2*10 + 1
ke_bloq = PotentialEnergy(num_elec, num_grid_each_dim)
cbloq = ke_bloq.decompose_bloq()
show_bloq(ke_bloq.decompose_bloq())